In [12]:
import ee
import geemap

# Authenticate to Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

first_year = 1985
last_year = 2020

data_folder = "projects/amazon-forest-regrowth/assets"
# This collection is not publicly accessible. To sign up for access,
# please see https://developers.planet.com/docs/integrations/gee/nicfi
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')

# Filter basemaps by date and get the first image from filtered results
basemap = nicfi.filter(ee.Filter.date('2016-03-01', '2016-07-01')).first()
basemap2 = nicfi.filter(ee.Filter.date('2018-03-01', '2018-07-01')).first()
basemap3 = nicfi.filter(ee.Filter.date('2020-03-01', '2020-07-01')).first()

mature = ee.Image(f"{data_folder}/mature_biomass")
agbd = ee.Image(f"{data_folder}/age_agbd").select('agbd')
ages = ee.Image(f"{data_folder}/age_agbd").select('age')
pantanal = ee.FeatureCollection(f"{data_folder}/raw/biomes_br") \
        .filter(ee.Filter.eq("CD_Bioma", 6)).geometry()

# import ages from MapBiomas
age = ee.Image(
    "projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_secondary_vegetation_age_v2"
).select("secondary_vegetation_age_2020")

# Load images from MapBiomas Collection 8 for Land Use Land Cover and Burned Area
lulc_raw = (
    ee.Image(
        "projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1"
    )
    .select([f"classification_{year}" for year in range(first_year, last_year + 1)])
    .byte()
    .rename([str(year) for year in range(first_year, last_year + 1)])
)

# restricting to only the land use in pixels classified as secondary forests in 2020:
# lulc_raw = lulc_raw.updateMask(age)
# List the categories that are DESIRED to be maintained
desired_values = ee.List([3, 6])
mask_all_ones = ee.List.repeat(1, desired_values.size())

# For each band, convert pixels with desired land use types to 1 - undesired types to zero
def remap_band(band_name):
    band = lulc_raw.select(ee.String(band_name))
    new_band = band.remap(desired_values, mask_all_ones, 0)
    return new_band.rename(ee.String(band_name))

# Map the function over the band names
remapped_image = lulc_raw.bandNames().map(remap_band)
# make mask by adding all pixels that add up to the total number of years (all pixels with desired categories)
remapped_image = ee.ImageCollection(remapped_image).toBands()
desired_mask = remapped_image.reduce("sum").eq(lulc_raw.bandNames().size().getInfo())

# age = age.updateMask(desired_mask).rename("age")
lulc = lulc_raw.updateMask(desired_mask)

# Visualization parameters
vis_planet = {'bands': ['R', 'G', 'B'], 'min': 64, 'max': 5454, 'gamma': 1.8}
vis_ages = {'min': 0, 'max': 35, 'palette': ['yellow', 'green']}
vis_agbd = {'min': 0, 'max': 300, 'palette': ['yellow', 'green']}
vis_mature = {'min': 0, 'max': 300, 'palette': ['blue', 'red']}
vis_geom = {
    'color': 'red',      # Color of the edge
    'width': 2           # Width of the edge
}

Map = geemap.Map()
Map.centerObject(basemap, 4)
# Map.addLayer(basemap, vis, '2016 mosaic')
Map.addLayer(basemap3, vis_planet, '2020 mosaic')
Map.addLayer(pantanal, vis_geom, 'pantanal')
Map.addLayer(ages, vis_ages, 'ages')
Map.addLayer(lulc, {}, 'lulc')
# Map.addLayer(agbd, vis_agbd, 'agbd')
Map.addLayer(mature, vis_mature, 'mature')
Map

Map(center=[0.0006306135584438607, -75.32069960585136], controls=(WidgetControl(options=['position', 'transpar…